In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('car-details.csv')
print(df.head())

                           name  company  model           edition  year  \
0        Maruti Swift Dzire VDI   Maruti  Swift         Dzire VDI  2014   
1  Skoda Rapid 1.5 TDI Ambition    Skoda  Rapid  1.5 TDI Ambition  2014   
2      Honda City 2017-2020 EXi    Honda   City     2017-2020 EXi  2006   
3     Hyundai i20 Sportz Diesel  Hyundai    i20     Sportz Diesel  2010   
4        Maruti Swift VXI BSIII   Maruti  Swift         VXI BSIII  2007   

    owner    fuel seller_type transmission  km_driven  mileage_mpg  engine_cc  \
0   First  Diesel  Individual       Manual     145500        55.00     1248.0   
1  Second  Diesel  Individual       Manual     120000        49.70     1498.0   
2   Third  Petrol  Individual       Manual     140000        41.60     1497.0   
3   First  Diesel  Individual       Manual     127000        54.06     1396.0   
4   First  Petrol  Individual       Manual     120000        37.84     1298.0   

   max_power_bhp   torque_nm  seats  selling_price  
0        

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6926 entries, 0 to 6925
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6926 non-null   object 
 1   company        6926 non-null   object 
 2   model          6926 non-null   object 
 3   edition        6926 non-null   object 
 4   year           6926 non-null   int64  
 5   owner          6926 non-null   object 
 6   fuel           6926 non-null   object 
 7   seller_type    6926 non-null   object 
 8   transmission   6926 non-null   object 
 9   km_driven      6926 non-null   int64  
 10  mileage_mpg    6718 non-null   float64
 11  engine_cc      6718 non-null   float64
 12  max_power_bhp  6717 non-null   float64
 13  torque_nm      6717 non-null   float64
 14  seats          6718 non-null   float64
 15  selling_price  6926 non-null   int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 865.9+ KB


In [4]:
df.isna().sum()

name               0
company            0
model              0
edition            0
year               0
owner              0
fuel               0
seller_type        0
transmission       0
km_driven          0
mileage_mpg      208
engine_cc        208
max_power_bhp    209
torque_nm        209
seats            208
selling_price      0
dtype: int64

In [5]:
df.shape

(6926, 16)

In [9]:
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].nunique()} unique values")
    print(df[col].unique())
    print(df[col].value_counts(normalize=True))

name: 2058 unique values
['Maruti Swift Dzire VDI' 'Skoda Rapid 1.5 TDI Ambition'
 'Honda City 2017-2020 EXi' ... 'Tata Nexon 1.5 Revotorq XT'
 'Ford Freestyle Titanium Plus Diesel BSIV'
 'Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV']
name
Maruti Swift Dzire VDI                          0.017037
Maruti Alto 800 LXI                             0.010973
Maruti Alto LXi                                 0.009962
Maruti Swift VDI                                0.008663
Maruti Swift VDI BSIV                           0.008085
                                                  ...   
Mahindra KUV 100 G80 K4 Plus                    0.000144
Hyundai Getz 1.1 GVS                            0.000144
Maruti 800 EX BSII                              0.000144
Datsun GO T Option                              0.000144
Toyota Innova 2.5 GX (Diesel) 8 Seater BS IV    0.000144
Name: proportion, Length: 2058, dtype: float64
company: 32 unique values
['Maruti' 'Skoda' 'Honda' 'Hyundai' 'Toyota' 'Ford' 'Renaul

In [10]:
df = df.drop(columns=['name', 'model','edition'])
df.head()

,company,year,owner,fuel,seller_type,transmission,km_driven,mileage_mpg,engine_cc,max_power_bhp,torque_nm,seats,selling_price
0,Maruti,2014,First,Diesel,Individual,Manual,145500,55.00,1248.0,74.00,190.000000,5.0,450000
1,Skoda,2014,Second,Diesel,Individual,Manual,120000,49.70,1498.0,103.52,250.000000,5.0,370000
2,Honda,2006,Third,Petrol,Individual,Manual,140000,41.60,1497.0,78.00,124.544455,5.0,158000
3,Hyundai,2010,First,Diesel,Individual,Manual,127000,54.06,1396.0,90.00,219.668960,5.0,225000
4,Maruti,2007,First,Petrol,Individual,Manual,120000,37.84,1298.0,88.20,112.776475,5.0,130000


In [11]:
df.duplicated().sum()   

np.int64(19)

In [12]:
df.drop_duplicates(inplace=True)
df.shape

(6907, 13)

In [13]:
df.duplicated().sum()

np.int64(0)

In [15]:
X = df.drop(columns=['selling_price'])
y = df['selling_price']

print(X.shape, y.shape)

(6907, 12) (6907,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5525, 12) (1382, 12) (5525,) (1382,)


In [22]:
num_cols = X_train.select_dtypes(include='number').columns.tolist()
cat_cols = X_train.select_dtypes(include='object').columns.tolist()
print("Numerical columns:", num_cols)
print("Categorical columns:", cat_cols)

Numerical columns: ['year', 'km_driven', 'mileage_mpg', 'engine_cc', 'max_power_bhp', 'torque_nm', 'seats']
Categorical columns: ['company', 'owner', 'fuel', 'seller_type', 'transmission']


In [23]:
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())])

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder',OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num',num_pipeline, num_cols),
    ('cat',cat_pipeline, cat_cols)
])

In [24]:
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [26]:
regressor = RandomForestRegressor(n_estimators=10,
                                  max_depth=5,
                                  random_state=42)

rf_model = Pipeline(steps=[
    ('pre', preprocessor),
    ('reg', regressor)
])

rf_model.fit(X_train, y_train)

,steps,"[('pre', ...), ('reg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [30]:
import numpy as np

# compute predictions and RMSE (use .values to avoid pandas index alignment issues)
y_train_pred = rf_model.predict(X_train)
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape, "y_train_pred shape:", y_train_pred.shape)
mse_train = mean_squared_error(y_train.values, y_train_pred)
train_rmse = np.sqrt(mse_train)
print(f'Train RMSE: {train_rmse:.2f}')

y_test_pred = rf_model.predict(X_test)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape, "y_test_pred shape:", y_test_pred.shape)
mse_test = mean_squared_error(y_test.values, y_test_pred)
test_rmse = np.sqrt(mse_test)
print(f'Test RMSE: {test_rmse:.2f}')


X_train shape: (5525, 12) y_train shape: (5525,) y_train_pred shape: (5525,)
Train RMSE: 169947.49
X_test shape: (1382, 12) y_test shape: (1382,) y_test_pred shape: (1382,)
Test RMSE: 172392.13
